In [1]:
# Setup imports
import os
import json
import numpy as np
import pandas as pd

import torch
from torch.utils import data

from config import Config
from pretrainer import PreTrainer
from m_psg2label import M_PSG2FEAT
from psg_dataset import PSG_pretrain_Dataset
from utils.utils_loss import age_loss
from utils.util_json import NumpyEncoder

np.random.seed(0)
torch.manual_seed(0)

In [2]:
# Setup data and model
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Config
config = Config()

# ds params
pretrain_params = {'batch_size': 2,
                       'num_workers': config.pre_n_workers,
                       'pin_memory': True}

# Setup data loaders
datasets_pre = PSG_pretrain_Dataset(config, 'test', n=10)
dataloaders_pre = data.DataLoader(datasets_pre, shuffle=False, **pretrain_params)
    
# Initialize network
model_F = M_PSG2FEAT(config).to(device)

# Pretraining and evaluation
config.save_dir = config.model_F_path
loss_fn = age_loss(device, gamma_cov=0.01)
pretrainer = PreTrainer(model_F, loss_fn, config,
                 device=device,
                 num_epochs=config.pre_max_epochs,
                 patience=config.pre_patience,
                 resume=True)


Number of recordings: 10
100%|██████████| 10/10 [00:00<00:00, 589.76it/s]
Data quality ensured.


In [4]:
# Run method
predictions = pretrainer.interpret_model(dataloaders_pre)


Computing model interpretation

  0%|          | 0/467 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 176.00 MiB (GPU 0; 8.00 GiB total capacity; 5.87 GiB already allocated; 70.25 MiB free; 49.60 MiB cached)